In [26]:
import requests
from urllib.request import urlopen
from urllib.parse import urlencode,unquote,quote_plus
import urllib
import lxml
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [27]:
판례목록_df = pd.read_html("법령검색목록.html")
판례목록_df = 판례목록_df[0]
판례목록_df

,순번,판례ID,제목,법원,사건유형,판결유형,선고일자
0,1,216815,가축분뇨배출시설변경허가신청불허가처분취소청구,대법원,대법원,일반행정,2021.06.30
1,2,216821,강도상해,대법원,대법원,형사,2021.06.30
2,3,216819,강제추행(인정된 죄명: 공연음란),대법원,대법원,형사,2021.06.30
3,4,216807,대여금,대법원,대법원,민사,2021.06.30
4,5,216803,손해배상(기),대법원,대법원,민사,2021.06.30
...,...,...,...,...,...,...,...
39995,39996,103550,거절사정,대법원,대법원,특허,1987.09.08
39996,39997,103535,건물철거,대법원,대법원,민사,1987.09.08
39997,39998,103544,건물철거등,대법원,대법원,민사,1987.09.08
39998,39999,103581,"공문서위조,공문서위조행사,사문서위조행사,공정증서원본불실기재행사,공정증서원본불실기재,...",대법원,대법원,형사,1987.09.08


In [28]:
info_df = pd.read_excel("id_판시사항,판결요지,참조조문,참조판례.xlsx")
info_df

,ID,판시사항,판결유형
0,216815,[1] 가축분뇨의 관리 및 이용에 관한 법률에 따른 가축분뇨 처리방법 변경허가가 허...,일반행정
1,216821,채권자를 폭행ㆍ협박하여 채무를 면탈함으로써 성립하는 강도죄에서 불법이득의사의 유무를...,형사
2,216819,"[1] 형사소송규칙 제142조 제1항, 제5항의 규정 취지 / 검사의 서면에 의한 ...",형사
3,216807,구 도시 및 주거환경정비법 제14조 제3항에서 일정 비율 이상의 토지 등 소유자의 ...,민사
4,216803,회생채권자가 제3자를 상대로 한 소송 계속 중에 회생채무자를 상대로 소송고지를 하고...,민사
...,...,...,...
39984,103550,가. 일본국 상표가 세계 여러 나라에 등록되어 있다하여 우리나라에서도 현저하게 인식...,특허
39985,103535,가. 부동산 매매계약에 있어서 매도인이 매수인에게 지체책임을 지워 매매계약을 해제하...,민사
39986,103544,권리남용의 성립요건권리의 행사가 권리의 남용에 해당한다고 할 수 있으려면 주관적으로...,민사
39987,103581,가. 진정성립을 인정한 검사작성의 피고인에 대한 피의자신문조서의 증거능력나. 수인 ...,형사


In [33]:
posts = info_df.get("판시사항")
posts

0        [1] 가축분뇨의 관리 및 이용에 관한 법률에 따른 가축분뇨 처리방법 변경허가가 허...
1        채권자를 폭행ㆍ협박하여 채무를 면탈함으로써 성립하는 강도죄에서 불법이득의사의 유무를...
2        [1] 형사소송규칙 제142조 제1항, 제5항의 규정 취지 / 검사의 서면에 의한 ...
3        구 도시 및 주거환경정비법 제14조 제3항에서 일정 비율 이상의 토지 등 소유자의 ...
4        회생채권자가 제3자를 상대로 한 소송 계속 중에 회생채무자를 상대로 소송고지를 하고...
                               ...                        
39984    가. 일본국 상표가 세계 여러 나라에 등록되어 있다하여 우리나라에서도 현저하게 인식...
39985    가. 부동산 매매계약에 있어서 매도인이 매수인에게 지체책임을 지워 매매계약을 해제하...
39986    권리남용의 성립요건권리의 행사가 권리의 남용에 해당한다고 할 수 있으려면 주관적으로...
39987    가. 진정성립을 인정한 검사작성의 피고인에 대한 피의자신문조서의 증거능력나. 수인 ...
39988    신뢰의 원칙에 비추어 운전사로서 사업상 주의의무를 해태하지 아니하였다고 본 사례직진...
Name: 판시사항, Length: 39989, dtype: object

In [34]:
# 불용어 리스트
# 불용어, 불필요 단어 제거
stop_words_df = pd.read_excel("stopwords.xlsx")
stop_words_df

,불용어
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
702,점
703,증
704,바
705,운


In [35]:
from konlpy.tag import Okt
okt = Okt()

oo = okt.pos(posts[0],
        norm=True,   # 정규화(normalization)
        stem=True    # 어간추출(stemming)
        )
print(oo)

[('[', 'Punctuation'), ('1', 'Number'), (']', 'Punctuation'), ('가축', 'Noun'), ('분뇨', 'Noun'), ('의', 'Josa'), ('관리', 'Noun'), ('및', 'Noun'), ('이용', 'Noun'), ('에', 'Josa'), ('관', 'Noun'), ('한', 'Josa'), ('법률', 'Noun'), ('에', 'Josa'), ('따르다', 'Verb'), ('가축', 'Noun'), ('분뇨', 'Noun'), ('처리', 'Noun'), ('방법', 'Noun'), ('변경', 'Noun'), ('허가', 'Noun'), ('가', 'Josa'), ('허가', 'Noun'), ('권', 'Suffix'), ('자의', 'Noun'), ('재량행위', 'Noun'), ('에', 'Josa'), ('해당', 'Noun'), ('하다', 'Verb'), ('여부', 'Noun'), ('(', 'Punctuation'), ('적극', 'Noun'), (')', 'Punctuation'), ('및', 'Noun'), ('가축', 'Noun'), ('분뇨', 'Noun'), ('처리', 'Noun'), ('방법', 'Noun'), ('변경', 'Noun'), ('불허', 'Noun'), ('가처분', 'Noun'), ('에', 'Josa'), ('대한', 'Noun'), ('사법심사', 'Noun'), ('의', 'Josa'), ('대상', 'Noun'), ('과', 'Josa'), ('판단', 'Noun'), ('기준', 'Noun'), ('[', 'Punctuation'), ('2', 'Number'), (']', 'Punctuation'), ('환경', 'Noun'), ('의', 'Josa'), ('훼손', 'Noun'), ('이나', 'Josa'), ('오염', 'Noun'), ('을', 'Josa'), ('발생', 'Noun'), ('시키다', 'Verb'), ('우려', 

In [36]:
from konlpy.tag import Mecab
# from eunjeon import Mecab
# 형태소 분석기 
tagger = Mecab()
stop_words = list(pd.read_excel("stopwords.xlsx").get("불용어"))
stop_words

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

In [37]:
def tokenizer_1(raw_texts, pos=["Noun","Alpha","Verb","Number"], stop_words=list(stop_words_df.get("불용어"))):
    p = okt.pos(raw_texts, 
            norm=True,   # 정규화(normalization)
            stem=True    # 어간추출(stemming)
            )
    o = [word for word, tag in p if len(word) > 1 and tag in pos and word[0] not in stop_words]
    return(o)

tokenizer_1(posts[80])


['재심',
 '청구',
 '받다',
 '법원',
 '당사자',
 '재심',
 '청구',
 '신청',
 '대하',
 '재판',
 '소극',
 '신청',
 '배척',
 '경우',
 '당사자',
 '고지',
 '소극',
 '재심',
 '청구',
 '받다',
 '법원',
 '인정',
 '형사소송법',
 '431조',
 '직권',
 '재심',
 '청구',
 '대한',
 '소송',
 '당사자',
 '신청',
 '당사자',
 '재심',
 '청구',
 '신청',
 '경우',
 '단지',
 '법원',
 '직권',
 '발동',
 '촉구',
 '법원',
 '신청',
 '대하',
 '재판',
 '필요',
 '설령',
 '법원',
 '신청',
 '배척',
 '당사자',
 '고지',
 '필요',
 '형사소송법',
 '420조',
 '431조']

In [38]:
def tokenizer_2(raw_texts, pos=["Noun","Alpha","Verb","Number"], stop_words=list(stop_words_df.get("불용어"))):
    nouns = []
  
    for noun in tagger.nouns(raw_texts):
        if noun not in stop_words and len(noun)>1:
            nouns.append(noun)
    return nouns

tokenizer_2(posts[80])

['재심',
 '청구',
 '법원',
 '당사자',
 '재심',
 '청구',
 '이유',
 '사실',
 '조사',
 '신청',
 '재판',
 '소극',
 '신청',
 '배척',
 '당사자',
 '고지',
 '소극',
 '재심',
 '청구',
 '법원',
 '필요',
 '인정',
 '형사',
 '소송법',
 '직권',
 '재심',
 '청구',
 '이유',
 '사실',
 '조사',
 '소송',
 '당사자',
 '조사',
 '신청',
 '당사자',
 '재심',
 '청구',
 '이유',
 '사실',
 '조사',
 '신청',
 '법원',
 '직권',
 '발동',
 '촉구',
 '의미',
 '법원',
 '신청',
 '재판',
 '필요',
 '법원',
 '신청',
 '배척',
 '당사자',
 '고지',
 '필요',
 '형사',
 '소송법']

In [39]:
vectorize = CountVectorizer(
    tokenizer=tokenizer_2, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=5,            # 단어가 출현하는 최소 문서의 개수
    stop_words= list(stop_words_df.get("불용어"))
)

X = vectorize.fit_transform(posts)
# X.toarray()
pd.DataFrame(X.toarray())

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['개의', '거니', '견지', '결과', '결론', '고려', '고용', '공동', '관계', '관련', '구체', '구토', '근거', '기관', '기점', '기준', '까닭', '나다', '나머지', '노동', '노동부', '누가', '대로', '도달', '도착', '동시', '따름', '때문', '마음', '방면', '부령', '부류', '불문', '비교', '사람', '상대', '생각', '서술', '시작', '시초', '아래', '아무', '연관', '의거', '의지', '이외', '이용', '이유', '일반', '일지', '입각', '입장', '정도', '제외', '종합', '주저', '즈음', '지경', '진짜', '필요', '한적', '할만', '항목', '행정', '형식'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


,0,1,2,3,4,5,6,7,8,9,...,10250,10251,10252,10253,10254,10255,10256,10257,10258,10259
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39984,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
count_df = pd.DataFrame(X.toarray())

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# 2번째와 3번째 문서(유클리디안 거리 값)
print(euclidean_distances(X[1], X[2]))#, X)
# 2번째와 3번째 문서(코사인유사도 값)
print(cosine_similarity(X[1], X[2]))#, X)
# [[0.1837195]]

[[56.50663678]]
[[0.07849698]]


In [41]:
# target_id 와 유사한 판례를 cosine_similarity으로 계산하여 유사벡터 도출
# 약 25초 소요
cosine_result = {}
target_id = 216815
x = info_df[info_df['ID']==target_id].index[0]  # target_id의 Id값의 index
for y in range(len(info_df)):
    cosine_result[info_df['ID'][y]] = cosine_similarity(X[x], X[y])[0][0]
sorted_cosine_dic = sorted(cosine_result.items(), key = lambda x:x[1], reverse = True)

# cosine_similariry 값 기준으로 내림차순 정렬된 유사 판례들
# 먼저나올수록 유사한 판례이다.
sorted_cosine_dic 

[(216815, 1.0000000000000018),
 (173157, 0.6130338074632523),
 (161708, 0.5965911654026868),
 (177477, 0.5771762897903082),
 (215147, 0.5747088857749061),
 (154390, 0.5689507277377293),
 (180199, 0.5337082216707446),
 (190771, 0.5115366375823214),
 (184828, 0.4921657856805038),
 (203017, 0.48946398040987393),
 (207090, 0.48480129376708053),
 (177332, 0.4756442985729686),
 (193350, 0.4714650811826009),
 (184531, 0.4699124725801967),
 (160418, 0.4588495820584292),
 (213789, 0.4491517987183846),
 (196396, 0.4432689007246804),
 (209909, 0.4337495931289017),
 (204513, 0.4282811539454802),
 (206096, 0.42506945548393843),
 (201051, 0.4233571604683759),
 (105455, 0.41751197562219444),
 (204626, 0.4168261797860388),
 (182956, 0.41193694013917503),
 (185882, 0.4011591667377293),
 (117005, 0.39520739054450155),
 (214459, 0.3885767173642542),
 (178873, 0.3853254862305651),
 (109571, 0.3620686571481455),
 (169207, 0.35674560497904706),
 (191203, 0.35603191677564344),
 (201221, 0.35465620960122146),

In [42]:
# target_id 에 해당하는 판례와 유사한 판례 상위 10개 도출
for id_ in sorted_cosine_dic[0:11]:
    url = "http://www.law.go.kr/DRF/lawService.do?OC=bmsong&target=prec&ID={}&type=HTML".format(id_[0])
    url2 = "https://www.law.go.kr/precInfoP.do?mode=0&precSeq={}&vSct=*".format(id_[0])
    print(url2)

https://www.law.go.kr/precInfoP.do?mode=0&precSeq=216815&vSct=*
https://www.law.go.kr/precInfoP.do?mode=0&precSeq=173157&vSct=*
https://www.law.go.kr/precInfoP.do?mode=0&precSeq=161708&vSct=*
https://www.law.go.kr/precInfoP.do?mode=0&precSeq=177477&vSct=*
https://www.law.go.kr/precInfoP.do?mode=0&precSeq=215147&vSct=*
https://www.law.go.kr/precInfoP.do?mode=0&precSeq=154390&vSct=*
https://www.law.go.kr/precInfoP.do?mode=0&precSeq=180199&vSct=*
https://www.law.go.kr/precInfoP.do?mode=0&precSeq=190771&vSct=*
https://www.law.go.kr/precInfoP.do?mode=0&precSeq=184828&vSct=*
https://www.law.go.kr/precInfoP.do?mode=0&precSeq=203017&vSct=*
https://www.law.go.kr/precInfoP.do?mode=0&precSeq=207090&vSct=*
